In [27]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata
import numpy as np

In [2]:
data = fetch_mldata('mnist original')

In [30]:
x, _y = data.data, data.target

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

In [42]:
sss = StratifiedShuffleSplit()
for train, test in sss.split(x,_y):
    x_train, y_train = x[train], _y[train]
    x_test, y_test = x[test], _y[test]
    
print(x_train.shape, y_train.shape)
x_train, x_test = np.reshape(x_train, newshape=(-1,28,28)), np.reshape(x_test, newshape=(-1,28,28))
print(x_train.shape, y_train.shape)

(63000, 784) (63000,)
(63000, 28, 28) (63000,)


In [26]:
n_steps = 28
n_inputs = 28
learning_rate = 0.001
n_epoch = 40

In [39]:
def next_batch(batch_size = 30, x_train = x_train, y_train = y_train):
    prev = 0
    for i in range(batch_size , x_train.shape[0], batch_size):
        yield x_train[prev:i], y_train[prev:i]
        prev = i

In [43]:
tf.reset_default_graph()

In [44]:
X = tf.placeholder(shape=[None, n_steps, n_inputs], dtype=tf.float32)
y = tf.placeholder(shape=[None], dtype=tf.int32)

In [45]:
bcell = tf.contrib.rnn.BasicRNNCell(num_units=50, activation=tf.nn.elu, dtype=tf.float32)

In [46]:
outputs, states = tf.nn.dynamic_rnn(cell=bcell, dtype=tf.float32, inputs=X)
logits = tf.layers.dense(units=10, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),inputs=states)

In [47]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels=y)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(k=1, targets=y, predictions=tf.nn.softmax(logits))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [48]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        batch = next_batch()
        for x_batch, y_batch in batch:
            sess.run(training_op, feed_dict = {X:x_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict = {X:x_train, y:y_train})
        acc_test = accuracy.eval(feed_dict = {X:x_test, y:y_test})
        print('epoch',epoch,'Training accuracy:',acc_train, 'Testing accuracy:',acc_test)

epoch 0 Training accuracy: 0.5910952 Testing accuracy: 0.58742857
epoch 1 Training accuracy: 0.7262381 Testing accuracy: 0.72842854
epoch 2 Training accuracy: 0.7994603 Testing accuracy: 0.8044286
epoch 3 Training accuracy: 0.819254 Testing accuracy: 0.82542855
epoch 4 Training accuracy: 0.830381 Testing accuracy: 0.8317143
epoch 5 Training accuracy: 0.8417778 Testing accuracy: 0.84814286
epoch 6 Training accuracy: 0.86131746 Testing accuracy: 0.85957146
epoch 7 Training accuracy: 0.8939365 Testing accuracy: 0.889
epoch 8 Training accuracy: 0.89749205 Testing accuracy: 0.89285713
epoch 9 Training accuracy: 0.90226984 Testing accuracy: 0.90014285
epoch 10 Training accuracy: 0.9050635 Testing accuracy: 0.9012857
epoch 11 Training accuracy: 0.9041905 Testing accuracy: 0.9021429
epoch 12 Training accuracy: 0.92138094 Testing accuracy: 0.9177143
epoch 13 Training accuracy: 0.9172698 Testing accuracy: 0.9161429
epoch 14 Training accuracy: 0.92825395 Testing accuracy: 0.9257143
epoch 15 Train